## BUILD FOR WST ETC with pyetc2 - IFS MAIN

In [1]:
#uncomment to clean up the variables
#%reset -f

In [2]:
### All done considering the IFS configuration

# (1) sky conditions -> to be computed with https://skycalc-ipy.readthedocs.io/en/stable/ > DONE
# need still to put the almanac 

# (2) with NDIT and DIT we will give also the SNR if needed - to dedide if it's a mean/median or else

# (3) Binning, especially in the spectral direction is missing

# (4) Not very relevant but for the optimal cases we can compute better which SNR to use for maximization

# (5) Line spread function should be put in the snr_from_source

# (6) togliere moon che é l'old parameter


#change the folder to your default installation folder
folder = '/Users/matteoferro/Library/Python/3.9/lib/python/site-packages/pyetc2/'

from pyetc2 import *
import os
import matplotlib.pyplot as plt
from astropy import constants
import skycalc_ipy
from mpdaf.obj import Spectrum
#from astropy import constants

import warnings
warnings.filterwarnings('ignore')

#this is a package of mine for plot quality, comment it and the following row in case you use this notebook 
from setplot import set_plot_style
set_plot_style()

#convenient function for the range check
def check_range(arr, M_min, M_max):
    
    if arr[0] > M_min:
        print('Trace starts after the first pixel!')
    if arr[-1] < M_max:
        print('Trace ends before the last pixel!')

def check_line(cen, fwhm, M_min, M_max):
    if cen > M_max:
        print('Line outside the last pixel!')
    elif cen + fwhm > M_max:
        print('Line near the last pixel!')
    if cen < M_min:
        print('Line outside the first pixel!')
    elif cen - fwhm < M_min:
        print('Line near the first pixel!')

In [3]:
### set up the WST container
obj_WST = WST(log = 'DEBUG', skip_dataload = False)

### set up the spectral calibration 
phot_system = PhotometricSystem()
sed_models = SEDModels()
filter_manager = FilterManager(phot_system)

########## build the observation
def build_obs_full(container_wst, fo):
    
    insfam = getattr(container_wst, fo["INS"]) 
    CONF = insfam[fo["CH"]]
    
    #work-around for the definition of the spectral type for the cont/line:
    
    if fo["SPEC"] == 'line':
        dummy_type = 'line'
    elif fo["SPEC"] in ('template', 'pl', 'bb'):
        dummy_type = 'cont'
        
    obs = dict(
    moon=fo["MOON"],
    airmass = fo["AM"],
    seeing = fo["SEE"],
    ndit = fo["NDIT"],
    dit = fo["DIT"],
    spec_type = dummy_type,
    spec_range_type = fo["SPEC_RANGE"],
    spec_range_kfwhm = fo["SPEC_KFWHM"],
    spec_range_hsize_spectels = fo["SPEC_HSIZE"],  
    ima_type = fo["OBJ"],
    ima_area = fo["IMA_AREA"],
    ima_aperture_type = fo["IMA_RANGE"],
    ima_kfwhm = fo["IMA_KFWHM"],
    ima_range_hsize_spaxels = fo["IMA_HSIZE"],
    skycalc = fo["SKYCALC"]
)
    
#compute the sky here if necessary, then we will move this into the actual pyetc, not here ! ! !
    if fo["SKYCALC"]:
        obs["skyemi"], obs["skyabs"] = get_sky2(container_wst, fo)
    
    container_wst.set_obs(obs)
    
    #dspec = dict(type='template', name='kc96/starb1', redshift=0., wave_center=4900, wave_width=100)
    #spec = container_wst.get_spec(CONF, dspec)
    
    #we compute the value here
    spec_input, spec =  get_spec2(container_wst, fo)
    
    #here the mpdaf image only in the resolved case
    if fo['OBJ'] == 'resolved':
        dima = dict(type=fo["IMA"], fwhm=fo["IMA_FWHM"], beta=fo["IMA_BETA"], n=fo["IMA_N"],
                reff=fo["IMA_REFF"], ell=fo["IMA_ELL"])
    
        ima = container_wst.get_ima(CONF, dima)
    else:
        ima = None
        
    old_flux = 1
        
    if (fo["OPT_SPEC"]) & (fo["SPEC_RANGE"] == "adaptative") & (dummy_type == "line"):
        container_wst.optimum_spectral_range(CONF, old_flux, ima, spec)

    if (fo["OPT_IMA"]) & (fo["IMA_RANGE"] == "circular_adaptative") & (fo["OBJ"] in ('ps', 'resolved')):
        A = CONF['lbda1']
        B = CONF['lbda2']
        f = fo['FRAC_SPEC_MEAN_OPT_IMAGE']

        delta = (1 - f) / 2
        AA = A + delta * (B - A)
        BB = B - delta * (B - A)
        
        #done with the mean SNR in a range defined around the center by the factor f
        container_wst.optimum_circular_aperture(CONF, old_flux, ima, spec, lrange=[AA,BB])
    
    return CONF, obs, spec, ima, spec_input

# # # # SPEC COMPUTATION 
def get_spec2(container_wst, fo, lsfconv = False, oversamp = 1):

    insfam = getattr(container_wst, fo["INS"]) 
    CONF = insfam[fo["CH"]]
    
    lstep = CONF['instrans'].get_step()
    l1,l2 = CONF['instrans'].get_start(), CONF['instrans'].get_end()
        
    if fo['SPEC'] == 'template':
        name, DEFAULT_WAVE, flux =  sed_models.template(f"{fo['TEMP_NAME']}.dat")
        redshift = fo['Z']
        band = fo['FIL']
        mag = fo['MAG']
        syst = fo['SYS']

        mag, syst = phot_system.auto_conversion(mag, band, syst)

        #redshift correction
        DEFAULT_WAVE *= (1+redshift)
        
        #check if it's all inside the range of the instrumental setup
        check_range(DEFAULT_WAVE, l1, l2)

        _, _, K = filter_manager.apply_filter(DEFAULT_WAVE, flux, band, mag, syst)

    elif fo['SPEC'] == 'bb':
        DEFAULT_WAVE = np.linspace(100,30000,10000)
        tmp = fo['TEMP']
        band = fo['FIL']
        mag = fo['MAG']
        syst = fo['SYS']
    
        flux =  sed_models.blackbody(DEFAULT_WAVE,tmp)

        mag, syst = phot_system.auto_conversion(mag, band, syst)

        _, _, K = filter_manager.apply_filter(DEFAULT_WAVE, flux, band, mag, syst)
    
    
    elif fo['SPEC'] == 'pl':
        DEFAULT_WAVE = np.linspace(100,30000,10000)
        indpl = fo['INDEX']
        band = fo['FIL']
        mag = fo['MAG']
        syst = fo['SYS']
        
        flux =  sed_models.powerlaw(DEFAULT_WAVE, indpl)

        mag, syst = phot_system.auto_conversion(mag, band, syst)

        _, _, K = filter_manager.apply_filter(DEFAULT_WAVE, flux, band, mag, syst)
    
    elif fo['SPEC'] == 'line':
        DEFAULT_WAVE = np.linspace(100,30000,10000)
        
        center = fo['INDEX']
        band = fo['FIL']
        mag = fo['MAG']
        syst = fo['SYS']
    
        center = fo['WAVE_CENTER']
        fwhm = fo['WAVE_FWHM']
        
        check_line(center, fwhm, l1, l2)

        tot_flux = fo['FLUX']

        flux = sed_models.gaussian_line(DEFAULT_WAVE, center, tot_flux, fwhm)
    
        K = 1
        oversamp = 10
        
    #put wave and flux*K in a MPDAF object here
    spec_raw = Spectrum(data=flux*K, wave=WaveCoord(cdelt=DEFAULT_WAVE[1]-DEFAULT_WAVE[0], crval=DEFAULT_WAVE[0]))
    
    # resample
    rspec = spec_raw.resample(lstep, start=l1)
    rspec = rspec.subspec(lmin=l1, lmax=l2)
    
    #this should be put in the snr_computation ! ! !
    if lsfconv:
        spec_calib = rspec.filter(width=CONF['lsfpix']) 
    else:
        spec_calib = rspec
    
    spec_calib.oversamp = oversamp
    
    #spettro di input in ingresso al telescopio e spettro convoluto e tagliato
    return spec_raw, spec_calib

# # # # # # FUNCTIONS FOR THE SKY COMPUTATION
def sun_moon_sep(fli):
    if not 0 <= fli <= 1:
        raise ValueError("FLI must be between 0 and 1.")
    theta_rad = np.arccos(1 - 2 * fli)  # result in radians
    theta_deg = np.degrees(theta_rad)  # convert to degrees
    return theta_deg

def compute_sky2(container_wst, fo):
    insfam = getattr(container_wst, fo["INS"]) 
    CONF = insfam[fo["CH"]]
    
    mss = sun_moon_sep(fo['FLI'])
    airmass = fo['AM']
    pwv = fo['PWV']
    allowed_pwv = [0.05, 0.01, 0.25, 0.5, 1.0, 1.5, 2.5, 3.5, 5.0, 7.5, 10.0, 20.0, 30.0]
    closest_value = min(allowed_pwv, key=lambda v: abs(v - pwv))

    if pwv not in allowed_pwv:
        print(f"PWV value not allowed, assigned the closest one: {pwv} → {closest_value}")
        pwv = closest_value
    
    skycalc = skycalc_ipy.SkyCalc()

    skycalc["msolflux"] = 130
    skycalc['observatory'] = 'lasilla'
    skycalc['airmass'] = airmass
    skycalc['pwv'] = pwv
    skycalc['moon_sun_sep'] = mss
    
    skycalc['wmin'] = CONF['lbda1']/10
    skycalc['wmax'] = CONF['lbda2']/10
    skycalc['wdelta'] = CONF['dlbda']/10
    ### we do it after so we can return also the unconvolved emission spectrum 
    #skycalc['lsf_type'] = 'Gaussian'
    #skycalc['lsf_gauss_fwhm'] = CONF['lsfpix']
    skycalc['wgrid_mode'] = 'fixed_wavelength_step'
    skycalc['wres'] = 20000 #not sure it is used somewhere, for now I keep it

    tbl = skycalc.get_sky_spectrum(return_type="tab-ext")
    
    if abs(tbl['lam'][0]*10 - CONF['lbda1'])>CONF['dlbda'] or \
       abs(tbl['lam'][-1]*10 - CONF['lbda2'])>CONF['dlbda'] or \
       abs(tbl['lam'][1]- tbl['lam'][0])*10 - CONF['dlbda']>0.01:
        raise ValueError(f'Incompatible bounds between called configuration and setup')
    
    d = dict()
    d['emi_orig'] = Spectrum(data = tbl['flux'], wave = CONF['wave'])
    d['emi'] = d['emi_orig'].filter(width=CONF['lsfpix'])
    d['abs'] = Spectrum(data = tbl['trans'], wave = CONF['wave'])
    
    return d, tbl

def get_sky2(container_wst, fo):
    d, _ = compute_sky2(container_wst, fo)
    return d['emi'], d['abs']    

In [4]:
sed_models.eso_spectra_files.keys()

dict_keys(['Pickles_O5V', 'Kinney_starb5', 'Kinney_starb4', 'Kinney_sa', 'Kinney_starb6', 'Pickles_O9V', 'Kinney_sb', 'Galev_E', 'Pickles_B2IV', 'Kinney_s0', 'Kinney_starb3', 'Kinney_starb2', 'Pickles_A0III', 'Kinney_starb1', 'Pickles_A0V', 'Pickles_K2V', 'Kurucz_B8V', 'Pickles_G0V', 'Pickles_M2V', 'Kurucz_G2V', 'Pickles_B9III', 'Kurucz_A1V', 'Pickles_B9V', 'Kurucz_B1V', 'Kurucz_F0V', 'Kinney_ell', 'Pickles_K7V', 'qso-interp'])

In [12]:
#SPEC = template, pl, bb, line

#TEMP_NAME is the name of the template, you can list them with sed_models.eso_spectra_files.keys()

################ only for line #############################
#SPEC_range = fixed, adaptative, None

#### only for fixed ####
#SPEC_HSIZE = 5 [range will be 2 * SPEC_HSIZE + 1]
########################

#### only for adaptative ####
#SPEC_KFWHM = 3 [range will be +- SPEC_KFWHM]
#OPT_SPEC = True, to compute the best SPEC_KFWHM for the maximization of the SNR
#############################
############################################################


#OBJ = sb, ps, resolved

################ only for resolved ###############################
#IMA = moffat, sersic 

#IMA_ELL = 0.

#### only for moffat ####
#IMA_FWHM = 1. in arcsec
#IMA_BETA = 2.5 in arcsec
#########################

#### only for sersic ####
#IMA_REFF = 1. in arcsec
#IMA_N = 2
#########################

##################################################################

################ only for sb ###############################
#IMA_AREA = 1. arcsec, area to use for the SNR computation
############################################################

################ only for ps and resolved ##################
#IMA_RANGE = circular_adaptative, square_fixed

#### only for square_fixed ####
#IMA_HSIZE = 5 [range will be 2 * IMA_HSIZE + 1]
###############################

#### only for circular_adaptative ####
#IMA_KFWHM = 3 [range will be +- IMA_KFWHM]
#OPT_IMA = True, to compute the best IMA_KFWHM for the maximization of the SNR
######################################
############################################################

### "SKYCALC": if false it uses the default configurations in the folder using MOON, 
###true it computes the sky with a call to the ESO Skycalc and want the FLI and PWV parameters

full_obs = {
    "INS": "ifs",
    "CH": "blue",
    
    "NDIT": 2.,
    "DIT": 1800.,    
    
    "MOON": 'greysky',
    "PWV": 3.5,
    "FLI": 0.5,
    "SEE": 1.,
    "AM": 1.0,
    "SKYCALC": False,
    
    "SPEC": 'template',
    "TEMP_NAME": 'Pickles_A0V',
    
    "MAG": 23,
    "SYS": 'AB',
    "FIL": 'rSDSS',
    "FLUX": 1e-17,
    
    "Z": 2.5,
    "TEMP": 3000.,
    "INDEX": -2.,
    "WAVE_CENTER": 5500,
    "WAVE_FWHM": 10,
    
    "OBJ": 'ps',
    
    "IMA": 'moffat',
    "IMA_ELL": 0.,
    
    "IMA_FWHM": 1.,
    "IMA_BETA": 2.5,
    
    "IMA_REFF": 1.,
    "IMA_N": 2,
        
    "SPEC_RANGE": 'adaptative',
    "SPEC_KFWHM": 3.,
    "SPEC_HSIZE": 5.,
    
    "IMA_AREA": 1.,
    "IMA_RANGE": 'circular_adaptative',
    "IMA_KFWHM": 3.,
    "IMA_HSIZE": 5.,
    
    "OPT_SPEC": False,
    "OPT_IMA": False,
    "FRAC_SPEC_MEAN_OPT_IMAGE": 0.8
}

In [13]:
CC, oo, ss, ii, ss_input = build_obs_full(obj_WST, full_obs)

Trace starts after the first pixel!


In [14]:
oo

{'moon': 'greysky',
 'airmass': 1.0,
 'seeing': 1.0,
 'ndit': 2.0,
 'dit': 1800.0,
 'spec_type': 'cont',
 'spec_range_type': 'adaptative',
 'spec_range_kfwhm': 3.0,
 'spec_range_hsize_spectels': 5.0,
 'ima_type': 'ps',
 'ima_area': 1.0,
 'ima_aperture_type': 'circular_adaptative',
 'ima_kfwhm': 3.0,
 'ima_range_hsize_spaxels': 5.0,
 'skycalc': False,
 'totexp': 1.0}

In [15]:
res = obj_WST.snr_from_source(CC, 1, ii, ss)

[DEBUG] Computing optimum values for kfwhm
[DEBUG] Optimum circular aperture nit=6 kfwhm=8.84 S/N=1.5 Aper=1.9 Flux=1.00e+00 Frac=0.85
[DEBUG] Optimum circular aperture nit=9 kfwhm=8.73 S/N=1.8 Aper=1.8 Flux=1.00e+00 Frac=0.91
[DEBUG] Optimum values of kfwhm at wavelengths edges: [8.84424471945754, 8.725713534556023]
[DEBUG] Computing frac and nspaxels for 20 wavelengths (lbin 20)
[DEBUG] Performing interpolation
[DEBUG] At 3700.0 A  FWHM: 0.95 Flux fraction: 0.85 Aperture: 1.9 Nspaxels: 45
[DEBUG] At 6100.0 A  FWHM: 0.83 Flux fraction: 0.91 Aperture: 1.8 Nspaxels: 37
[DEBUG] Source type ps & cont Flux 1.00e+00 S/N 4.7 FracFlux 0.885 Nspaxels 43 Nspectels 1


In [19]:
res

{'cube': {'trunc_spec': <Spectrum(shape=(3751,), unit='', dtype='float64')>},
 'input': {'atm_abs': <Spectrum(shape=(3751,), unit='', dtype='>f8')>,
  'ins_trans': <Spectrum(shape=(3751,), unit='', dtype='float64')>,
  'atm_emi': <Spectrum(shape=(3751,), unit='', dtype='>f8')>,
  'flux_source': <Spectrum(shape=(3751,), unit='', dtype='float64')>,
  'dl': 0.64,
  'flux': 1,
  'moon': 'greysky',
  'dit': 1800.0,
  'ndit': 2.0,
  'airmass': 1.0,
  'spec_type': 'cont',
  'ima_type': 'ps'},
 'spec': {'snr': <Spectrum(shape=(3751,), unit='', dtype='float64')>,
  'snr_mean': 4.653672548072596,
  'snr_max': 8.397768217019676,
  'snr_min': 0.0,
  'snr_med': 4.486063590479423,
  'nph_source': <Spectrum(shape=(3751,), unit='', dtype='float64')>,
  'nph_sky': <Spectrum(shape=(3751,), unit='', dtype='float64')>,
  'frac_ima': <Spectrum(shape=(3751,), unit='', dtype='float64')>,
  'nb_spaxels': array([45, 45, 45, ..., 37, 37, 37]),
  'nb_voxels': array([45, 45, 45, ..., 37, 37, 37]),
  'nb_spectels'